In [1]:
# import all relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime
from datetime import timedelta
import math
import random
import timeit

In [2]:
import urllib.request, json 

In [3]:
# duisburg data
# we simply send the API the request parameter city with the value 132, which is duisburgs city ID
# city IDs can be found here https://api.nextbike.net/maps/nextbike-live.json?list_cities=1
with urllib.request.urlopen("https://api.nextbike.net/maps/nextbike-live.json?city=132") as url:
    duisburg_stations_data = json.loads(url.read().decode())

In [4]:
# muelheim data
with urllib.request.urlopen("https://api.nextbike.net/maps/nextbike-live.json?city=137") as url:
    muelheim_stations_data = json.loads(url.read().decode())

In [5]:
# now we will go through the data, which is internally now stored as dictionary
# we will save all stations data in a list called duisburg_city_stations
# we also delete some key/value pairs, because they store lists, which will we cannot display in a dataframe
# notice that we only use stations, which have 'spot' set to true
# as stated in nextbikes API documentation this means that this data is indeed a stations
# places with 'spot' set to false, are locations where people left their bike unauthorised
duisburg_city_stations = []
for place in duisburg_stations_data['countries'][0]['cities'][0]['places']:
    if place['spot'] == True:
        del place['bike_list']
        del place['bike_numbers']
        del place['bike_types']
        duisburg_city_stations.append(place)

In [6]:
# same as above but for muelheim
muelheim_stations = []
for place in muelheim_stations_data['countries'][0]['cities'][0]['places']:
    if place['spot'] == True:
        del place['bike_list']
        del place['bike_numbers']
        del place['bike_types']
        muelheim_stations.append(place)

In [7]:
# now we will pass the data to dataframes
muelheim_stations = pd.DataFrame(muelheim_stations)
duisburg_city_stations = pd.DataFrame(duisburg_city_stations)

In [8]:
# first we will join the latitude and longitude to one coordinates column
muelheim_stations['coordinates'] = list(zip(muelheim_stations['lat'].round(4), muelheim_stations['lng'].round(4)))
duisburg_city_stations['coordinates'] = list(zip(duisburg_city_stations['lat'].round(4), duisburg_city_stations['lng'].round(4)))

In [9]:
# we drop all columns except for coordinates and number
# number is nextbikes internal id of a station, so we will use it as index
muelheim_stations.drop(muelheim_stations.columns.difference(['coordinates', 'number']), 1, inplace=True)
duisburg_city_stations.drop(duisburg_city_stations.columns.difference(['coordinates', 'number']), 1, inplace=True)
muelheim_stations.set_index('number', inplace=True)
duisburg_city_stations.set_index('number', inplace=True)

In [12]:
# this will work as soon as the cleaned data is saved somewhere, so we can read it here again.
# this will visualize both stations dataframe in a nice way

Du_map = folium.Map(location=(51.44, 6.83), #took the middle of limits as coordinates
                       zoom_start=10, control_scale=True, max_zoom=20)
Du_map.add_child(plugins.HeatMap(duisburg["dest"], radius=15)) #adding heatmap points with the cleaned orig coordinates
# Du_map.add_child(plugins.HeatMap(duisburg["dest"], radius=15)) #adding heatmap points with the cleaned dest coordinates
muelheim_stations_list = list(muelheim_stations['coordinates'])
duisburg_city_stations = list(duisburg_city_stations['coordinates'])
positions = muelheim_stations_list + marburg_stations_list
for position in positions:
    folium.CircleMarker(radius=5, location=position, 
                                 color='black', fill_color='black').add_to(Du_map)
Du_map #showing Map

NameError: name 'duisburg' is not defined

In [13]:
# we now concat our data, so we have one dataframe which we will use for duisburgs data
duisburg_stations = pd.concat([muelheim_stations, duisburg_city_stations])

In [14]:
duisburg_stations.index.rename('id', inplace=True)

In [15]:
duisburg_stations

,coordinates
id,
7901,"(51.4225, 6.8759)"
7903,"(51.4299, 6.8861)"
7904,"(51.428, 6.8845)"
7910,"(51.4269, 6.8801)"
7912,"(51.425, 6.8818)"
...,...
7429,"(51.4284, 6.8004)"
7428,"(51.3849, 6.8002)"
7439,"(51.4208, 6.7841)"


marburg

In [16]:

with urllib.request.urlopen("https://api.nextbike.net/maps/nextbike-live.json?city=438") as url:
    marburg_stations_data = json.loads(url.read().decode())

In [17]:
marburg_stations = []
for place in marburg_stations_data['countries'][0]['cities'][0]['places']:
    if place['spot'] == True:
        del place['bike_list']
        del place['bike_numbers']
        del place['bike_types']
        marburg_stations.append(place)

In [18]:
marburg_stations = pd.DataFrame(marburg_stations)
marburg_stations['coordinates'] = list(zip(marburg_stations['lat'].round(4), marburg_stations['lng'].round(4)))

In [19]:
marburg = pd.read_csv("../data/marburg_clean.csv")

In [21]:
type(marburg['orig'][0])

str

In [20]:
Ma_map = folium.Map(location=(50.81, 8.74), #took the middle of limits as coordinates
                       zoom_start=10, control_scale=True, max_zoom=20)
Ma_map.add_child(plugins.HeatMap(marburg["orig"], radius=15)) #adding heatmap points with the cleaned orig coordinates
Ma_map.add_child(plugins.HeatMap(marburg["dest"], radius=15)) #adding heatmap points with the cleaned dest coordinates
marburg_stations_list = list(marburg_stations['coordinates'])
positions = marburg_stations_list 
for position in positions:
    folium.CircleMarker(radius=5, location=position, 
                                 color='black', fill_color='black').add_to(Ma_map)
    
    
Ma_map

ValueError: Location should consist of two numerical values, but '(' of type <class 'str'> is not convertible to float.